# 02 – Data Cleaning & Feature Engineering  
**CRISP-DM Phase 3: Data Preparation**  
This notebook cleans, standardises and feature-engineers the raw data so it’s ready for modeling and analysis.

### Objectives
* Load the raw sample CSV from `inputs/datasets/raw/HousePricesRecords.csv`.
* Handle nulls and duplicates.
* Parse dates → `Year` / `Month`.
* Map and one-hot encode categorical fields:  
  – `Old/New` → binary flag  
  – `Duration` → binary flag (Freehold vs Leasehold)  
  – `Property Type` → four flags (`Property_D`, `Property_F`, `Property_S`, `Property_T`)  
* Save the fully cleaned, feature-engineered table for downstream use.

### Inputs
* `inputs/datasets/raw/HousePricesRecords.csv`  

### Outputs
* `outputs/datasets/collection/HousePricesRecords_clean.csv`  

### Additional Comments  
#### Business Requirements Addressed  
* **BR1**: Ensures high-quality data for the Overview & Correlation pages.  
* **BR2**: Creates the features needed for statistical tests on property attributes.  

#### Additional Notes  
* This cleaned CSV is the single source of truth for Notebooks 3–5 and all Streamlit pages.  

---

### Import Required Libraries  
This cell brings in the key Python packages we’ll use throughout the data‐cleaning notebook:  
- **pandas** (`pd`) for reading, manipulating, and saving tabular data.  
- **os** for interacting with the file system (checking/creating folders, constructing file paths).  


In [1]:
import pandas as pd
import os
import numpy as np

#### Check DataFrame Dimensions  
Use `.shape` to quickly see how many rows and columns were loaded into your sample. This confirms you’ve successfully read in 1,000 rows and shows the total number of features available for cleaning and analysis.


In [2]:
input_path = "../inputs/datasets/raw/price_paid_records.csv"
chunksize = 200_000    # tweak up/down depending on your RAM
topn      = 1_000      # how many newest rows to keep
latest = None

In [3]:
for chunk in pd.read_csv(
    input_path,
    usecols=[
        "Price","Date of Transfer","Property Type","Old/New",
        "Duration","Town/City","County","PPDCategory Type"
    ],
    parse_dates=["Date of Transfer"],
    chunksize=chunksize,
    low_memory=False
):
    # for this chunk, keep its top `topn` newest rows
    chunk_top = chunk.nlargest(topn, "Date of Transfer")

    if latest is None:
        latest = chunk_top
    else:
        # merge with previous bests, then re-take the top `topn`
        latest = pd.concat([latest, chunk_top]).nlargest(topn, "Date of Transfer")
    

df_chunk = latest.reset_index(drop=True)

In [4]:
df_chunk = latest.reset_index(drop=True)

Preview the first 5 rows to see if it is working 

In [5]:
df_chunk.head()

,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,County,PPDCategory Type
0,277000,2017-06-29,S,N,F,WICKFORD,ESSEX,A
1,30000,2017-06-29,F,N,L,HULL,CITY OF KINGSTON UPON HULL,A
2,551000,2017-06-29,T,N,F,CHISLEHURST,GREATER LONDON,A
3,240000,2017-06-29,S,N,F,BEDFORD,BEDFORD,B
4,527500,2017-06-29,D,N,F,HEMEL HEMPSTEAD,HERTFORDSHIRE,B


---


#### Inspect Column Data Types  
Print out the data type of each column in your sample to verify that numbers, dates, and text fields have been correctly inferred. This helps identify which columns may need conversions (e.g., strings → datetime or numeric types) before cleaning.


In [6]:
print("Column Data Types:")
print(df_chunk.dtypes)

Column Data Types:
Price                        int64
Date of Transfer    datetime64[ns]
Property Type               object
Old/New                     object
Duration                    object
Town/City                   object
County                      object
PPDCategory Type            object
dtype: object


---

#### Examine Category Distributions  
This cell loops over key categorical fields to display the unique values and their frequencies. By inspecting `value_counts()`, you can:

- Spot typos or inconsistent labels (e.g. “Freehold” vs “freehold”).  
- Identify rare categories that may need grouping or special handling.  
- Confirm which categories to one-hot encode in the next step.

In [7]:
categorical_cols = ["Property Type", "Old/New", "Duration", "County", "Town/City"]
for col in categorical_cols:
    print(f"\nValue counts for {col}:")
    print(df_chunk[col].value_counts())


Value counts for Property Type:
Property Type
S    286
T    282
D    239
F    140
O     53
Name: count, dtype: int64

Value counts for Old/New:
Old/New
N    992
Y      8
Name: count, dtype: int64

Value counts for Duration:
Duration
F    828
L    172
Name: count, dtype: int64

Value counts for County:
County
GREATER LONDON      79
KENT                48
HAMPSHIRE           39
DEVON               39
ESSEX               29
                    ..
BLACKPOOL            1
WARRINGTON           1
MIDDLESBROUGH        1
POWYS                1
ISLE OF ANGLESEY     1
Name: count, Length: 103, dtype: int64

Value counts for Town/City:
Town/City
LONDON             51
BRISTOL            23
NOTTINGHAM         15
POOLE              12
PLYMOUTH           11
                   ..
KNEBWORTH           1
STAMFORD            1
BISHOP AUCKLAND     1
PENZANCE            1
CAERNARFON          1
Name: count, Length: 437, dtype: int64


#### Identify & Remove Rows with Missing Price or Date  
This cell first prints the number of null values in each column to highlight data completeness issues. It then drops any rows where **`Price`** or **`Date of Transfer`** is missing—since these fields are critical for analysis—and reports the new DataFrame shape.


In [8]:

print("Missing values in each column:")
print(df_chunk.isnull().sum())

df_chunk.dropna(subset=["Price", "Date of Transfer"], inplace=True)
print(f"Shape after dropping missing price/date: {df_chunk.shape}")

Missing values in each column:
Price               0
Date of Transfer    0
Property Type       0
Old/New             0
Duration            0
Town/City           0
County              0
PPDCategory Type    0
dtype: int64
Shape after dropping missing price/date: (1000, 8)


---

#### Drop Irrelevant Columns  
This cell removes columns that aren’t needed for our analysis or modelling, helping to streamline the dataset:

- **`Transaction unique identifier`**: a non-informative ID field.  
- **`District`**: redundant given the `County` and `Town/City` columns.  
- **`Record Status - monthly file only`**: metadata relevant only to monthly file management.  

The `errors='ignore'` flag ensures the code won’t fail if any of these columns are already absent.


In [9]:
df_chunk.drop(columns=[
    "Transaction unique identifier",
    "District",
    "Record Status - monthly file only"
], inplace=True, errors='ignore')

#### Convert “Date of Transfer” to datetime  
This cell casts the `Date of Transfer` column from text to pandas `datetime64[ns]`, enabling time-based operations (e.g., extracting year/month, filtering by date).  
- `pd.to_datetime(..., errors='coerce')` attempts to parse each entry; any invalid or unrecognisable strings become `NaT` (Not a Time).  


In [10]:
df_chunk["Date of Transfer"] = pd.to_datetime(df_chunk["Date of Transfer"], errors='coerce')

#### Extract Year & Month Features  
This cell creates two new columns—`Year` and `Month`—by extracting these components from the `Date of Transfer` datetime. These features allow us to analyse and model seasonality or temporal trends in sale prices.


In [11]:
df_chunk["Year"] = df_chunk["Date of Transfer"].dt.year
df_chunk["Month"] = df_chunk["Date of Transfer"].dt.month

In [12]:
max_date    = df_chunk["Date of Transfer"].max()
cutoff_date = max_date - pd.DateOffset(years=3)

# 2) filter to only those transfers within the last 3 years
before_rows = df_chunk.shape[0]
df_chunk    = df_chunk[df_chunk["Date of Transfer"] >= cutoff_date]
after_rows  = df_chunk.shape[0]

print(
    f"Filtered out {before_rows - after_rows} rows; "
    f"remaining {after_rows} rows from {cutoff_date.date()} → {max_date.date()}"
)

Filtered out 0 rows; remaining 1000 rows from 2014-06-29 → 2017-06-29


#### Encode Key Categorical Features  
This cell turns our categorical flags into numeric form so they can be used by the regression model:

- **Map binary flags**  
  - `Old/New`: maps `'N' → 0` (existing) and `'Y' → 1` (new build)  
  - `Duration`: maps `'F' → 1` (freehold) and `'L' → 0` (leasehold)  
- **One-hot encode**  
  - `Property Type`: creates separate `Property_<Type>` columns (e.g., `Property_Detached`, `Property_Flat`, etc.) with 0/1 indicators  


In [13]:
df_chunk["Old/New"] = df_chunk["Old/New"].map({'N': 0, 'Y': 1})
df_chunk["Duration"] = df_chunk["Duration"].map({'F': 1, 'L': 0})
df_chunk = pd.get_dummies(df_chunk, columns=["Property Type"], prefix="Property")

#### Remove Duplicate Records & Reset Index  
This cell ensures data integrity by dropping any duplicate rows—so each property sale appears only once—and then resets the DataFrame index to a clean, sequential range.

In [14]:
df_chunk.drop_duplicates(inplace=True)
df_chunk.reset_index(drop=True, inplace=True)

### Geographical Aggregates (using Town/City)

#### Define a Unified Region Feature  
This line creates a new column, `Region`, by copying the values from the existing `Town/City` field.  
Having a dedicated `Region` column makes subsequent grouping and aggregation (e.g., median price by region) more intuitive and self-documenting.

In [15]:
df_chunk["Region"] = df_chunk["Town/City"]

#### Compute & Merge Regional Median Prices  
This block groups the data by `Region`, calculates the median sale price for each region (`RegionMedianPrice`), and then merges that information back into `df_chunk` so each record includes its region’s median price.

In [16]:
region_median = (
    df_chunk
    .groupby("Region")["Price"]
    .median()
    .rename("RegionMedianPrice")
)
df_chunk = df_chunk.merge(region_median, on="Region", how="left")

#### Compute & Merge Regional Sale Counts  
This block counts the number of transactions for each `Region` (`RegionSaleCount`) and then joins that count back into `df_chunk`, so each record knows how many sales occurred in its region.

In [17]:
region_counts = (
    df_chunk["Region"]
    .value_counts()
    .rename("RegionSaleCount")
)
df_chunk = df_chunk.join(region_counts, on="Region")

#### Target Encoding & Count Encoding for County  
To capture local market effects without exploding one-hot dimensions

Target-encode the `County` by computing the median sale price per county and merging it in as `CountyMedianPrice`.  

In [18]:
county_median = (
    df_chunk
    .groupby("County")["Price"]
    .median()
    .rename("CountyMedianPrice")
)
df_chunk = df_chunk.merge(county_median, on="County", how="left")

Count-encode the `County` by counting how many transactions occurred in each county and merging it in as `CountySaleCount`.

In [19]:
county_counts = (
    df_chunk["County"]
    .value_counts()
    .rename("CountySaleCount")
)
df_chunk = df_chunk.join(county_counts, on="County")

### Price Transformations & Outlier Handling

To stabilise variance and reduce the effect of extreme sale prices on our model,

#### Log-transform the target  
This step applies a natural log (plus one) to the `Price` column, creating a new `LogPrice` feature. Log transformation helps stabilise variance and reduce right‐skew in the target distribution.


In [20]:
df_chunk["LogPrice"] = np.log1p(df_chunk["Price"])

####  Compute the 99.5th percentile cutoff  
Here we calculate the value at the 99.5th percentile of `LogPrice`. This threshold flags the most extreme high‐price outliers for removal.


In [21]:
upper_cutoff = df_chunk["LogPrice"].quantile(0.995)

#### Remove outliers and report  
This section filters out any rows where `LogPrice` exceeds the cutoff, then prints how many rows were dropped and the DataFrame’s new shape—ensuring that extreme values don’t unduly influence the model.

In [22]:
before_rows = df_chunk.shape[0]
df_chunk = df_chunk[df_chunk["LogPrice"] <= upper_cutoff]
after_rows = df_chunk.shape[0]

print(f"Dropped {before_rows - after_rows} outlier rows; new shape: {df_chunk.shape}")

Dropped 5 outlier rows; new shape: (989, 20)


---

#### Save Cleaned Data to CSV  
This cell creates the output directory (if it doesn’t already exist), writes the fully cleaned DataFrame to a CSV file, and prints a confirmation message. The resulting file is the single source of truth for downstream analysis and the Streamlit app.


In [23]:
output_path = '../outputs/datasets/collection/HousePricesRecords_clean.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_chunk.to_csv(output_path, index=False)
print(f"Cleaned data saved to: {output_path}")

Cleaned data saved to: ../outputs/datasets/collection/HousePricesRecords_clean.csv
